In [31]:
import pandas as pd
import numpy as np
import csv
import json
from scipy.stats import linregress
from statsmodels.stats.weightstats import ztest as ztest

# local code
from source.interactive_plots import interactive_linear_regression_calibration_plot
from source.get_elements      import get_elements
from source.outliers          import dixon_test

Load data from XRF analyses

In [32]:
xrf_data = pd.read_csv("../data/interim/xrf_data_clean.csv") # load xrf data
# drop uncertainty columns 
xrf_data.drop([column for column in xrf_data.columns if column.endswith("+/-")], axis=1, inplace=True)

# Calibration

Load data for standard reference materials and clean

In [33]:
srm_data = pd.read_csv("../data/interim/standard_reference_material_certified_values.csv") # load SRM data
srm_data.drop(["1SD", "95% Confidence Low", "95% Confidence High"], axis=1, inplace=True) # drop unnecessary columns

# clean SRM data
for row in srm_data.iterrows(): 
    row[1]["Sample ID"] = row[1]["Sample ID"].lower()
    row[1]["Analyte"] = row[1]["Analyte"].split(",",1)[0]

    # clean certified value
    cert_val = row[1]["Certified Value"]
    if cert_val.startswith("<"): 
        row[1]["Certified Value"] = float(cert_val.lstrip("< ")) / 2 # replace BDL with half value
    else:
        row[1]["Certified Value"] = float(row[1]["Certified Value"])

    # clean units
    row[1]["Units"] = row[1]["Units"].lstrip("(").rstrip(")")
    if row[1]["Units"] == "wt.%":  # convert units
       row[1]["Certified Value"] = row[1]["Certified Value"] * 1e4
       row[1]["Units"] = "ppm"

## Remove standard data for a given element that is unsuitable for calibration

Compare the measured concentration of each element for a given standard to the distribution of measured concentrations for that element across all non-standards. If the measured concentration 

    1.) falls outside the range of the distribution 
    2.) is more than three standard deviations from the mean 

that value is set to `NaN` (all values set to `NaN` are filtered out during calibration). 

Note: I should consider conducting a test to determine if each distribution is normally or log-normally distributed; if the latter, I should log-transform the data and then calculate the z-score. 

In [34]:
def z_score(data, value): 
    mean = np.mean(data)
    std = np.std(data)

    return (value - mean) / std

In [35]:
non_standards_data = xrf_data.loc[xrf_data["qaqc_type"]!="standard"]

outlier_stddev_cutoff = 5

for standard in xrf_data.loc[xrf_data["qaqc_type"]=="standard", "sample_id"].unique(): 
    for date in xrf_data.loc[xrf_data["sample_id"]==standard, "date"].unique():
        for element in get_elements(xrf_data.columns.to_list()): 
            standard_date_element = xrf_data.loc[(xrf_data["sample_id"]==standard) & (xrf_data["date"]==date)][element].values

            if (non_standards_data[element] > standard_date_element[0]).all() | \
               (non_standards_data[element] < standard_date_element[0]).all(): 
                
                # print(non_standards_data[element].to_numpy())
                z = z_score(non_standards_data[element], standard_date_element[0])
                if abs(z) > outlier_stddev_cutoff: 
                    # print(standard + ", " + date + ", " + element + ", " + str(standard_date_element[0]) + ", " + str(z))

                    xrf_data.loc[(xrf_data["sample_id"]==standard) & (xrf_data["date"]==date), element] = np.nan

Get the elements analyzed by the XRF and for which concentrations are reported for one or more standard reference materials

In [36]:
elements = get_elements(
                        list(
                            set(srm_data["Analyte"].unique()) & \
                            set(xrf_data.columns.to_list())
                            )
                        )

Construct a linear regression model for each element in order to predict the true concentration from the measured concentration

In [37]:
# initialize dictionary to hold lin. reg. models for each element
reg = {}
for element in elements: 

    # get IDs of standard reference materials
    srm = srm_data.loc[srm_data["Analyte"]==element]["Sample ID"].unique() 

    ## TRAIN 
    # limit training data to standards for which we have standard reference material info for the element at hand
    data_train = xrf_data.loc[(xrf_data["qaqc_type"]=="standard") & (xrf_data["sample_id"].isin(srm))]
    data_train = data_train.dropna(subset=[element]) # change to true condition statement

    x_train = [srm_data.loc[
                            (srm_data["Sample ID"]==sample) & \
                            (srm_data["Analyte"]==element)
                            ]["Certified Value"].values[0] for sample in data_train["sample_id"]]
    x_train = np.array(x_train)
    y_train = data_train[element].to_numpy()

    model = linregress(x_train, y_train) # fit linear regression model

    if model.slope != 0: #only use calibration curve if meaningful (i.e., if variance in dep. var. explained by variance in indep. var.)
        # invert model so that measured concentration is independent var. and true concentration is dependent var. (i.e., y = m*x + b --> x = (1/m)*y - (b/m))
        intercept_inv = -model.intercept / model.slope
        slope_inv = model.slope ** -1

        ## PREDICT (i.e., calibrate)
        data_predict = xrf_data.loc[xrf_data["qaqc_type"]!="standard"]
        data_predict = data_predict.dropna(subset=[element])

        x_predict = data_predict[element]
        x_predict = x_predict.to_numpy()
        y_predict = slope_inv * x_predict + intercept_inv

        ## Save model results
        reg[element] = {} # initialize empty dict to save model results

        reg[element]["model"]                 = model
        reg[element]["x_train"]               = x_train
        reg[element]["y_train"]               = y_train
        reg[element]["score"]                 = model.rvalue
        reg[element]["y-intercept std error"] = model.intercept_stderr
        reg[element]["slope_inv"]             = slope_inv
        reg[element]["intercept_inv"]         = intercept_inv
        reg[element]["x_predict"]             = x_predict
        reg[element]["y_predict"]             = y_predict
        
        xrf_data.loc[xrf_data["qaqc_type"]!="standard", element] = y_predict

/Users/robertcollar/miniconda3/envs/EPSC-552-Mont-St-Hilaire/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/Users/robertcollar/miniconda3/envs/EPSC-552-Mont-St-Hilaire/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning:

divide by zero encountered in double_scalars



Display the results of the calibration

In [38]:
dropdown_buttons = {
    "data": 
        {
            "name": "Elements", 
            "columns": list(reg.keys())
        }
    }

interactive_linear_regression_calibration_plot(dropdown_buttons, reg, x_axis_label="True concentration (ppm)", y_axis_label="Measured concentration (ppm)", title="")

Calculate the detection limit for each element. 

In [39]:
for element in elements: 
    detection_limit = reg[element]["slope_inv"] * (reg[element]["model"].intercept + 3*reg[element]["y-intercept std error"]) + reg[element]["intercept_inv"]
    reg[element]["detection_limit"] = detection_limit

Apply detection limit to dataset.

In [40]:
for element in reg.keys(): 
    limit = reg[element]["detection_limit"]
    xrf_data[element].where(xrf_data[element] >= limit, other=f"<{limit}", inplace=True)

# Evaluating Precision

In [41]:
lab_duplicates = xrf_data[xrf_data["qaqc_type"]=="lab duplicate"]

lab_parent_indexes = []

for row in lab_duplicates.iterrows(): 
    lab_dup_id = row[1]["sample_id"]
    lab_parent_id = lab_dup_id.rstrip('L')

    condition = xrf_data["sample_id"] == lab_parent_id
    index = xrf_data.index[condition][0]
    lab_parent_indexes.append(index)

lab_parents = xrf_data.iloc[lab_parent_indexes]

lab_pairs = pd.concat([lab_duplicates, lab_parents])

In [42]:
def half_detection_limit(x): 
    if isinstance(x, str): 
        if x.startswith("<"): 
            x = float(x.lstrip("< ")) / 2 # replace BDL with half value
    return x

In [43]:
lab_pair_diffs = lab_parents.copy()

elements_dup = get_elements(xrf_data.columns.to_list())

lab_pair_diffs.loc[:, elements_dup] = lab_pair_diffs.loc[:, elements_dup].apply(pd.to_numeric, errors="coerce")
lab_duplicates = lab_duplicates.copy()
lab_duplicates.loc[:, elements_dup] = lab_duplicates.loc[:, elements_dup].apply(pd.to_numeric, errors="coerce")

for row in lab_pair_diffs.iterrows():
    parent_id = row[1]["sample_id"]
    duplicate_id = parent_id + "L"

    parent_entry = row[1].to_frame().transpose()
    parent_entry.reset_index(inplace=True)
    parent_values = parent_entry.loc[:, elements_dup]

    duplicate_entry = lab_duplicates.loc[lab_duplicates["sample_id"]==duplicate_id]
    duplicate_entry.reset_index(inplace=True)
    duplicate_values = duplicate_entry.loc[:, elements_dup]
    
    condition = lab_pair_diffs["sample_id"] == parent_id
    lab_pair_diffs.loc[condition, elements_dup] = parent_values.subtract(duplicate_values).values

    parent_values = parent_values.applymap(half_detection_limit)
    duplicate_values = duplicate_values.applymap(half_detection_limit)
    pair_values = pd.concat((parent_values, duplicate_values))

    condition = xrf_data["sample_id"] == parent_id
    xrf_data.loc[condition, elements_dup] = pair_values.mean().values

    condition = xrf_data["sample_id"] == duplicate_id
    xrf_data.drop(xrf_data[condition].index, inplace=True)
xrf_data.reset_index(inplace=True)

lab_pair_diffs = lab_pair_diffs[["sample_id"] + elements_dup]

In [44]:
def std_dev_from_pairs(diffs): 
    diffs = np.asarray(diffs)
    n = diffs.size
    
    std_dev = np.sqrt(np.sum(np.square(diffs)) / (2 * n))

    return std_dev, n

In [199]:
def rel_std_dev(element, data, std_dev): 
    data = data.applymap(half_detection_limit)
    mean = data.loc[xrf_data["qaqc_type"] != "standard", element].mean()
    rsd = 100 * std_dev / mean

    return rsd, mean

In [200]:
rel_std_dev("Ca", xrf_data, 1393.85)

(7.365442474333678, 18924.185544278465)

In [202]:
analytical_precision = {}
xrf_data_bdl_replaced = xrf_data.applymap(half_detection_limit)
for element in elements_dup: 
    diffs = lab_pair_diffs[element]
    diffs.dropna(inplace=True)
    if (len(diffs) >=3) & (False in list(lab_pair_diffs[element].isnull())): # min. for Dixon's Q Test & remove elements w/ all null values
        diffs = dixon_test(list(diffs), left=True, right=True, confidence_level=95)[0]
    std_dev, n = std_dev_from_pairs(diffs)
    
    mean = xrf_data_bdl_replaced.loc[xrf_data_bdl_replaced["qaqc_type"] != "standard", element].mean()
    rsd = 100 * std_dev / mean

    if not np.isnan(std_dev): 
        analytical_precision[element] = {}
        analytical_precision[element]["standard deviation"] = std_dev
        analytical_precision[element]["number of pairs"] = n
        analytical_precision[element]["rsd"] = rsd
        analytical_precision[element]["mean"] = mean

/var/folders/s5/y_27w4ms6tj21xxv0z3tg3cw0000gn/T/ipykernel_4712/812834540.py:5: RuntimeWarning:

invalid value encountered in double_scalars



In [66]:
with open("../data/interim/analytical_precision.json", "w") as outfile: 
    json.dump(analytical_precision, outfile, indent=4)

In [59]:
field_duplicates = xrf_data[xrf_data["qaqc_type"]=="field duplicate"]
field_parent_indexes = []

for row in field_duplicates.iterrows(): 
        field_dup_id = row[1]["sample_id"]
        field_parent_id = field_dup_id.rstrip('F')

        condition = xrf_data["sample_id"] == field_parent_id
        index = xrf_data.index[condition][0]
        field_parent_indexes.append(index)
print(field_parent_indexes)
field_parents = xrf_data.iloc[field_parent_indexes]

field_pairs = pd.concat([field_duplicates, field_parents])

[43, 90, 38, 162, 99, 153, 57, 94, 186, 124, 165, 170]


In [75]:
field_pair_diffs
xrf_data[["U"]].head(n=50)

,U
0,<4.808030478491732
1,<4.808030478491732
2,<4.808030478491732
3,10.3
4,<4.808030478491732
5,9.7
6,<4.808030478491732
7,6.9
8,<4.808030478491732
9,<4.808030478491732


In [67]:

elements_dup = get_elements(xrf_data.columns.to_list())

field_pair_diffs = field_parents.copy()
field_pair_diffs.loc[:, elements_dup] = field_pair_diffs.loc[:, elements_dup].apply(pd.to_numeric, errors="coerce")

field_duplicates = field_duplicates.copy()
field_duplicates.loc[:, elements_dup] = field_duplicates.loc[:, elements_dup].apply(pd.to_numeric, errors="coerce")

for row in field_pair_diffs.iterrows():
    parent_id = row[1]["sample_id"]
    duplicate_id = parent_id + "F"

    parent_entry = row[1].to_frame().transpose()
    parent_entry.reset_index(inplace=True)
    parent_values = parent_entry.loc[:, elements_dup]

    duplicate_entry = field_duplicates.loc[field_duplicates["sample_id"]==duplicate_id]
    duplicate_entry.reset_index(inplace=True)
    duplicate_values = duplicate_entry.loc[:, elements_dup]
    
    condition = field_pair_diffs["sample_id"] == parent_id
    field_pair_diffs.loc[condition, elements_dup] = parent_values.subtract(duplicate_values).values

    # parent_values = parent_values.applymap(half_detection_limit)
    # duplicate_values = duplicate_values.applymap(half_detection_limit)
    # pair_values = pd.concat((parent_values, duplicate_values))

    # condition = xrf_data["sample_id"] == parent_id
    # xrf_data.loc[condition, elements_dup] = pair_values.mean().values

    # condition = xrf_data["sample_id"] == duplicate_id
    # xrf_data.drop(xrf_data[condition].index, inplace=True)
# xrf_data.reset_index(inplace=True)

field_pair_diffs = field_pair_diffs[["sample_id"] + elements_dup]

In [76]:
field_heterogeneity = {}
for element in elements_dup: 
    diffs =field_pair_diffs[element]
    diffs.dropna(inplace=True)
    if (len(diffs) >=3) & (False in field_pair_diffs["Cd"].isnull()): # min. for Dixon's Q Test & remove elements w/ all null values
        diffs = dixon_test(list(diffs), left=True, right=True, confidence_level=95)[0]
    std_dev, n = std_dev_from_pairs(diffs)

    if not np.isnan(std_dev): 
        field_heterogeneity[element] = {}
        field_heterogeneity[element]["standard deviation"] = analytical_precision[element]["standard deviation"]
        field_heterogeneity[element]["heterogeneity"] = std_dev - analytical_precision[element]["standard deviation"]
        field_heterogeneity[element]["number of pairs"] = n

/var/folders/s5/y_27w4ms6tj21xxv0z3tg3cw0000gn/T/ipykernel_4712/812834540.py:5: RuntimeWarning:

invalid value encountered in double_scalars



In [78]:
with open("../data/interim/field_heterogeneity.json", "w") as outfile: 
    json.dump(field_heterogeneity, outfile, indent=4)

In [ ]:
# alpha= 0.05
# element = "Ba"
# normality_test_data = xrf_data.loc[xrf_data["qaqc_type"]!="standard", element]
# k, p = stats.normaltest(normality_test_data)
# if p < alpha:
#     print("reject null hypothesis that sample is normall distributed")

In [79]:
# export data to csv file
xrf_data.to_csv('../data/interim/xrf_data_calib.csv')

In [21]:

    # xrf_data.to_excel('../data/interim/xrf_data_calib.xlsx', na_rep="NaN")

In [205]:
dataset_evaluation = dict.fromkeys(get_elements(xrf_data.columns))
for element in dataset_evaluation.keys(): 
    dataset_evaluation[element] = {"calibration":          {}, \
                                   "analytical precision": {}, \
                                   "field heterogeneity":  {}}

for element in reg.keys(): 
    dataset_evaluation[element]["calibration"]          = {"R-squared":          round(reg[element]["score"],                               2)}

for element in analytical_precision.keys(): 
    dataset_evaluation[element]["analytical precision"] = {"standard deviation": round(analytical_precision[element]["standard deviation"], 2), \
                                                           "rsd":                round(analytical_precision[element]["rsd"],                2), \
                                                           "number of pairs":    round(analytical_precision[element]["number of pairs"],    2)}

for element in field_heterogeneity.keys(): 
    dataset_evaluation[element]["field heterogeneity"]  = {"heterogeneity":      round(field_heterogeneity[element]["heterogeneity"],       2), \
                                                           "number of pairs":    round(field_heterogeneity[element]["number of pairs"],     2)}   

In [229]:
discard = {}
usable_dataset_evaluation = dataset_evaluation.copy()
for element in dataset_evaluation.keys(): 
    if dataset_evaluation[element]["calibration"] == {}:
        discard[element] = "srm values missing"

    elif dataset_evaluation[element]["calibration"]["R-squared"] <= 0.3:
        discard[element] = "poor calibration"

    elif dataset_evaluation[element]["analytical precision"] == {}: 
        discard[element] = "insufficient data for analytical precision"
        
    elif dataset_evaluation[element]["analytical precision"]["number of pairs"] < 9:
        discard[element] = "insufficient data for analytical precision"
    
    elif dataset_evaluation[element]["analytical precision"]["rsd"] > 30:
        discard[element] = "RSD is too high"

for element in discard.keys():
    usable_dataset_evaluation.pop(element)


In [232]:
discard

{'Ag': 'poor calibration',
 'Au': 'srm values missing',
 'Bi': 'poor calibration',
 'Cd': 'poor calibration',
 'Ce': 'poor calibration',
 'Cl': 'srm values missing',
 'Co': 'insufficient data for analytical precision',
 'Cr': 'insufficient data for analytical precision',
 'Hg': 'srm values missing',
 'Ni': 'poor calibration',
 'Pd': 'srm values missing',
 'Pr': 'poor calibration',
 'Pt': 'srm values missing',
 'Rh': 'srm values missing',
 'S': 'insufficient data for analytical precision',
 'Sb': 'RSD is too high',
 'Se': 'poor calibration',
 'Sm': 'poor calibration',
 'Sn': 'insufficient data for analytical precision',
 'Zr': 'poor calibration'}

In [231]:
usable_dataset_evaluation


{'As': {'calibration': {'R-squared': 0.97},
  'analytical precision': {'standard deviation': 5.1,
   'rsd': 27.41,
   'number of pairs': 27},
  'field heterogeneity': {'heterogeneity': -0.34, 'number of pairs': 12}},
 'Ba': {'calibration': {'R-squared': 1.0},
  'analytical precision': {'standard deviation': 11.42,
   'rsd': 2.58,
   'number of pairs': 26},
  'field heterogeneity': {'heterogeneity': 35.16, 'number of pairs': 12}},
 'Ca': {'calibration': {'R-squared': 1.0},
  'analytical precision': {'standard deviation': 1393.85,
   'rsd': 7.37,
   'number of pairs': 27},
  'field heterogeneity': {'heterogeneity': 723.91, 'number of pairs': 12}},
 'Cu': {'calibration': {'R-squared': 1.0},
  'analytical precision': {'standard deviation': 2.37,
   'rsd': 10.01,
   'number of pairs': 26},
  'field heterogeneity': {'heterogeneity': 0.79, 'number of pairs': 11}},
 'Fe': {'calibration': {'R-squared': 0.99},
  'analytical precision': {'standard deviation': 1307.87,
   'rsd': 2.69,
   'number o

In [100]:
dataset_evaluation

{'Ag': {'calibration': {'R-squared': 0.04},
  'analytical precision': {'standard deviation': 846.45,
   'number of pairs': 22},
  'field heterogeneity': {'heterogeneity': 141.3, 'number of pairs': 10}},
 'As': {'calibration': {'R-squared': 0.04},
  'analytical precision': {'standard deviation': 846.45,
   'number of pairs': 22},
  'field heterogeneity': {'heterogeneity': 141.3, 'number of pairs': 10}},
 'Au': {'calibration': {'R-squared': 0.04},
  'analytical precision': {'standard deviation': 846.45,
   'number of pairs': 22},
  'field heterogeneity': {'heterogeneity': 141.3, 'number of pairs': 10}},
 'Ba': {'calibration': {'R-squared': 0.04},
  'analytical precision': {'standard deviation': 846.45,
   'number of pairs': 22},
  'field heterogeneity': {'heterogeneity': 141.3, 'number of pairs': 10}},
 'Bi': {'calibration': {'R-squared': 0.04},
  'analytical precision': {'standard deviation': 846.45,
   'number of pairs': 22},
  'field heterogeneity': {'heterogeneity': 141.3, 'number of